In [1]:
from ipywebrtc import CameraStream, ImageRecorder, VideoStream, VideoRecorder
import time
import librosa
import numpy as np
from PIL import Image
import IPython.display as ipd
import av

In [2]:
camera_without_audio = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 640, 'height': 480 }
                       })


camera_with_audio = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': True,
                       'video': { 'width': 640, 'height': 480 }
                       })

recorder = VideoRecorder(stream=camera_with_audio)
camera_without_audio

CameraStream(constraints={'facing_mode': 'user', 'audio': False, 'video': {'width': 640, 'height': 480}})

In [10]:
record_time = 5  # in seconds
buffer_time = 1 # in seconds

recorder.recording = True
time.sleep(buffer_time)  # one second buffer

print('RECORDING HAS STARTED')

for i in range(record_time):
    print(f'ends in {record_time - i} seconds')
    time.sleep(1)
print('RECORDING DONE')

time.sleep(buffer_time)  # one second buffer
recorder.recording = False

original_time_length = record_time + buffer_time * 2

RECORDING HAS STARTED
ends in 5 seconds
ends in 4 seconds
ends in 3 seconds
ends in 2 seconds
ends in 1 seconds
RECORDING DONE


In [11]:
recorder.save('video.webm')

In [23]:
audio, fs = librosa.load('video.webm', sr=None)
audio = audio[fs*buffer_time:-fs*buffer_time]

/home/tk/.virtualenvs/webcam/lib/python3.6/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [24]:
len(audio) / fs

4.9

In [26]:
ipd.Audio(audio, rate=fs) # load a local WAV file

In [27]:
container = av.open('video.webm')

average_rate = container.streams.video[0].average_rate
fps = average_rate.numerator / average_rate.denominator
arrays = []

for frame in container.decode(video=0):
#     frame.to_image().save('frame-%04d.jpg' % frame.index)
    array = frame.to_ndarray(format='rgb24')
    arrays.append(array)
    
arrays = arrays[round(fps*buffer_time):-round(fps*buffer_time)]

In [28]:
len(arrays) / fps

4.851